In [2]:
import glob
import os
from csv import reader
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
import numpy as np
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import itertools

In [ ]:
#function that goes through all the sub-folders up to the final files

def percorrePasta(pasta):
    files = []
    for f in glob.glob(pasta):
        if(os.path.isdir(f + "/")):
            f = f + "/*"
            files += percorrePasta(f)
        else:
            if(os.path.splitext(f)[1] == ".nii." ):
                files += [f]
    return files

arr = percorrePasta('DS1_scans/*')
print(arr)

In [ ]:
#all the exams

dictImageID = {}

with open('total_total.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        dictImageID[row[13]] = {"id": row[0], "sex": row[2], "deltames":row[4], "age": row[6], "mmse":row[7], 
                                "cdr":row[8], "protocol":row[12]}

idPaciente = 0
lastFound = []

df = pd.DataFrame([],columns=['nr_paciente','paciente', 'sexo', 'tecnica','data', 
                              'visita', 'visita_id', 'idade','MMSE', 'Global CDR','protocol' ,'nome']) 

for found in arr:
    splitted = found.split('/')
    splitUnderline = splitted[5].split('_')
    imageID = splitUnderline[len(splitUnderline)-1].replace(".nii", "")
    imageID = imageID.replace("I","")
    user = dictImageID[imageID]
    dMes = user["deltames"].split(' ')
    visita_id = 0
    if(dMes[1] == "Month"):
        visita_id = dMes[2]
    if lastFound != [] and lastFound[1] != splitted[1]:
        idPaciente+=1

    df.loc[df.shape[0]] = [idPaciente, splitted[1], user["sex"], splitted[2], splitted[3], 
                           user["deltames"], visita_id, user["age"],user["mmse"],user["cdr"], 
                           user["protocol"], splitted[5] ]
    
    lastFound = splitted
    
#df.to_csv('results.csv', index=False)

In [ ]:
#dataset with no repeats of exams in the same month, and no patients with only one exam.
dictImageID = {}

with open('total_total.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        dictImageID[row[13]] = {"id": row[0], "sex": row[2], "deltames":row[4], "age": row[6], "mmse":row[7], 
                                "cdr":row[8], "protocol":row[12], "image-id":row[13], "tecnica":row[3],"exames":[]}

def getImageID(splitted):
    splitUnderline = splitted[5].split('_')
    imageID = splitUnderline[len(splitUnderline)-1].replace(".nii", "")
    imageID = imageID.replace("I","")

    return imageID
        
pacientes = {}
idPaciente = 0

for found in arr:
    splitted = found.split('/')
    imageID = getImageID(splitted)
    user = dictImageID[imageID]
    dMes = user["deltames"].split(' ')
    visita_id = 0
    if(dMes[1] == "Month"):
        visita_id = dMes[2]
        
    if(user["id"] not in pacientes):
        pacientes[user["id"]] = {"sex":user["sex"],"tecnica":splitted[2],"protocol":user["protocol"], "exames":[]}
        pacientes[user["id"]]["exames"].append({"data":splitted[3], "dmes": visita_id, "image-id": imageID, 
                                                "age":user["age"],"mmse":user["mmse"],"cdr":user["cdr"]})
    else:
        pacientes[user["id"]]["exames"].append({"data":splitted[3], "dmes": visita_id, "image-id": imageID, 
                                                "age":user["age"],"mmse":user["mmse"],"cdr":user["cdr"]})    
                
    
dgg = pd.DataFrame([],columns=['nr_paciente','paciente', 'sexo', 'tecnica','data', 
                              'visita', 'visita_id', 'idade','MMSE', 'Global CDR','protocol' ,'nome']) 

removidos = 0
for key, value in list(pacientes.items()):
    arrDmes = []
    toRemove = []
    for exame in pacientes[key]["exames"]:
        if(exame["dmes"] in arrDmes):
            toRemove.append(exame)
        else:
            arrDmes.append(exame["dmes"])
            
    for exame in toRemove:
        pacientes[key]["exames"].remove(exame)
    
    if(len(value["exames"])<=1):
        del pacientes[key]
        
                
dgg = pd.DataFrame([],columns=['nr_paciente','paciente', 'sexo', 'tecnica','data', 
                              'visita_id', 'idade','MMSE', 'Global CDR','protocol' ,'nome']) 
print(pacientes)

idPaciente = 0
for key, value in list(pacientes.items()):
    for exame in value["exames"]:
        dgg.loc[dgg.shape[0]] = [idPaciente, key, value["sex"], value["tecnica"], exame["data"],
                                exame["dmes"], exame["age"], exame["mmse"], exame["cdr"], value["protocol"],
                                exame["image-id"]]
    idPaciente+=1
#print(pacientes)

dgg.to_csv('results1.csv', index=False)

In [5]:
#dataset where one row is a patient, and everything else is in columns.

'''
As colunas 'visita_0', 'visita_6','visita_12','visita_24','visita_36','visita_48' são para prencher
apenas com 0's e 1's, no caso de existir ou não.
Todas as outras colunas, é para prencher com os respetivos valores.

'''

dictImageID = {}

with open('total_total.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    for row in csv_reader:
        dictImageID[row[13]] = {"id": row[0], "sex": row[2], "deltames":row[4], "age": row[6], "mmse":row[7], 
                                "cdr":row[8], "protocol":row[12], "image-id":row[13], "tecnica":row[3],"exames":[]}

def getImageID(splitted):
    splitUnderline = splitted[5].split('_')
    imageID = splitUnderline[len(splitUnderline)-1].replace(".nii", "")
    imageID = imageID.replace("I","")

    return imageID

pacientes = {}

for found in arr:
    splitted = found.split('/')
    imageID = getImageID(splitted)
    user = dictImageID[imageID]
    dMes = user["deltames"].split(' ')
    visita_id = 0
    if(dMes[1] == "Month"):
        visita_id = dMes[2]
        
    if(user["id"] not in pacientes):
        pacientes[user["id"]] = {"sex":user["sex"],"tecnica":splitted[2],"protocol":user["protocol"], "exames":[]}
        pacientes[user["id"]]["exames"].append({"data":splitted[3], "dmes": visita_id, "image-id": imageID, 
                                                "age":user["age"],"mmse":user["mmse"],"cdr":user["cdr"]})
    else:
        pacientes[user["id"]]["exames"].append({"data":splitted[3], "dmes": visita_id, 
                                                "image-id": imageID, 
                                                "age":user["age"],"mmse":user["mmse"],"cdr":user["cdr"]})    
                
    
dgg = pd.DataFrame([],columns=['nr_paciente','paciente', 'sexo', 'tecnica','data', 
                              'visita', 'visita_id', 'idade','MMSE', 'Global CDR','protocol' ,'nome']) 

for key, value in list(pacientes.items()):
    arrDmes = []
    toRemove = []
    for exame in pacientes[key]["exames"]:
        if(exame["dmes"] in arrDmes):
            toRemove.append(exame)
        else:
            arrDmes.append(exame["dmes"])
            
    for exame in toRemove:
        pacientes[key]["exames"].remove(exame)
    
    if(len(value["exames"])<=1):
        del pacientes[key]
                
dff = pd.DataFrame([],columns=['nr_paciente','paciente', 'sexo', 'tecnica', 'protocol',
                               'visita_0', 'visita_6','visita_12','visita_18','visita_24','visita_36','visita_48',
                               'idade_0', 'idade_6','idade_12','idade_18','idade_24','idade_36','idade_48',
                               'MMSE_0', 'MMSE_6','MMSE_12','MMSE_18','MMSE_24','MMSE_36','MMSE_48',
                               'CDR_0', 'CDR_6','CDR_12','CDR_18','CDR_24','CDR_36','CDR_48',
                               'imagem_0','imagem_6','imagem_12','imagem_18','imagem_24','imagem_36','imagem_48']) 

idPaciente = 0
    
    
for key, value in list(pacientes.items()):
    visitas = [[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],
               [0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' ']]
    #print(visitas)
    for exame in value["exames"]:
        visita = {"id":1,"idade":exame["age"],"mmse":exame["mmse"], "cdr":exame["cdr"], 
                  "imagem-id":exame["image-id"]}
                    
        
        if(exame["dmes"] == "6"): 
            visitas[1] = list(visita.values())
        elif(exame["dmes"] == "12"): 
            visitas[2] = list(visita.values())     
        elif(exame["dmes"] == "18"): 
            visitas[3] = list(visita.values())
        elif(exame["dmes"] == "24"): 
            visitas[4] = list(visita.values())
        elif(exame["dmes"] == "36"): 
            visitas[5] = list(visita.values())
        elif(exame["dmes"] == "48"): 
            visitas[6] = list(visita.values())
        else:
            visitas[0] = list(visita.values())
    
    transposta = list(map(lambda *i: [j for j in i], *visitas))
    transpostaEmLinha = []
    for x in transposta:
        transpostaEmLinha+=x
        
    
    dff.loc[dff.shape[0]] = [idPaciente, key, value["sex"], value["tecnica"], 
                             value["protocol"]] + transpostaEmLinha
    idPaciente+=1

#print(pacientes)

dff.to_csv('results_final.csv', index=False)
dff.to_excel (r'results2_novo.xlsx', index = None, header=True)

In [ ]:
def contaDelta(arrBin):
    n = [0, 0, 0, 0, 0, 0]
    
    #Seis
    if arrBin[0] == 1 and arrBin[1] == 1: n[0] += 1
    if arrBin[1] == 1 and arrBin[2] == 1: n[0] += 1
    if arrBin[2] == 1 and arrBin[3] == 1: n[0] += 1
    if arrBin[3] == 1 and arrBin[4] == 1: n[0] += 1
    
    #Doze
    if arrBin[0] == 1 and arrBin[2] == 1: n[1] += 1
    if arrBin[1] == 1 and arrBin[3] == 1: n[1] += 1
    if arrBin[2] == 1 and arrBin[4] == 1: n[1] += 1
    if arrBin[4] == 1 and arrBin[5] == 1: n[1] += 1
    if arrBin[5] == 1 and arrBin[6] == 1: n[1] += 1

    #Dezoito
    if arrBin[0] == 1 and arrBin[3] == 1: n[2] += 1
    if arrBin[1] == 1 and arrBin[4] == 1: n[2] += 1
    if arrBin[3] == 1 and arrBin[5] == 1: n[2] += 1
    
    #VinteEQuatro
    if arrBin[0] == 1 and arrBin[4] == 1: n[3] += 1
    if arrBin[2] == 1 and arrBin[5] == 1: n[3] += 1
    if arrBin[4] == 1 and arrBin[6] == 1: n[3] += 1
    
    #Trinta e Seis
    if arrBin[0] == 1 and arrBin[5] == 1: n[4] += 1
    if arrBin[2] == 1 and arrBin[6] == 1: n[4] += 1
    
    #Quarenta e Oito
    if arrBin[0] == 1 and arrBin[6] == 1: n[5] += 1
    
    return n

def arrayToBin(arr):
    newArr = [0, 0, 0, 0, 0, 0]
    iNewArr = 0
    for x in arr:
        if x != 0:
            newArr[iNewArr] = 1
        iNewArr+=1
    return newArr

iguais = {}

for combination in itertools.product(range(2), repeat=6):
    iguais[''.join(map(str, combination))] = 0

for key, value in list(pacientes.items()):
    visitas = [[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],
               [0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' ']]
    #print(visitas)
    for exame in value["exames"]:
        visita = {"id":1,"idade":exame["age"],"mmse":exame["mmse"], "cdr":exame["cdr"], 
                  "imagem-id":exame["image-id"]}
        
        if(exame["dmes"] == "6"): 
            visitas[1] = list(visita.values())
        elif(exame["dmes"] == "12"): 
            visitas[2] = list(visita.values())     
        elif(exame["dmes"] == "18"): 
            visitas[3] = list(visita.values())
        elif(exame["dmes"] == "24"): 
            visitas[4] = list(visita.values())
        elif(exame["dmes"] == "36"): 
            visitas[5] = list(visita.values())
        elif(exame["dmes"] == "48"): 
            visitas[6] = list(visita.values())
        else:
            visitas[0] = list(visita.values())
    
    transposta = list(map(lambda *i: [j for j in i], *visitas))
    transpostaEmLinha = []
    for x in transposta:
        transpostaEmLinha+=x
    
    arrToPrint = [visitas[0][0], visitas[1][0], visitas[2][0], visitas[3][0], visitas[4][0], visitas[5][0],
                 visitas[6][0]]
    
    combinacoes = contaDelta(arrToPrint)
    combinacoesBin = arrayToBin(combinacoes)
    combinacoesBinStr = ''.join(map(str, combinacoesBin))
    #if combinacoesBinStr == '000000':
        #print(key)
    iguais[combinacoesBinStr]+=1
    
#print(pacientes.keys())
#print(iguais)
    
for ig, value in iguais.items():
    print(ig + ': ' + str(value))

In [ ]:
#LIST OF PATIENTS BY PERIOD

def contaDelta(arrBin):
    n = [0, 0, 0, 0, 0, 0]
    
    #Seis
    if arrBin[0] == 1 and arrBin[1] == 1: n[0] += 1
    if arrBin[1] == 1 and arrBin[2] == 1: n[0] += 1
    if arrBin[2] == 1 and arrBin[3] == 1: n[0] += 1
    if arrBin[3] == 1 and arrBin[4] == 1: n[0] += 1
    
    #Doze
    if arrBin[0] == 1 and arrBin[2] == 1: n[1] += 1
    if arrBin[1] == 1 and arrBin[3] == 1: n[1] += 1
    if arrBin[2] == 1 and arrBin[4] == 1: n[1] += 1
    if arrBin[4] == 1 and arrBin[5] == 1: n[1] += 1
    if arrBin[5] == 1 and arrBin[6] == 1: n[1] += 1

    #Dezoito
    if arrBin[0] == 1 and arrBin[3] == 1: n[2] += 1
    if arrBin[1] == 1 and arrBin[4] == 1: n[2] += 1
    if arrBin[3] == 1 and arrBin[5] == 1: n[2] += 1
    
    #VinteEQuatro
    if arrBin[0] == 1 and arrBin[4] == 1: n[3] += 1
    if arrBin[2] == 1 and arrBin[5] == 1: n[3] += 1
    if arrBin[4] == 1 and arrBin[6] == 1: n[3] += 1
    
    #Trinta e Seis
    if arrBin[0] == 1 and arrBin[5] == 1: n[4] += 1
    if arrBin[2] == 1 and arrBin[6] == 1: n[4] += 1
    
    #Quarenta e Oito
    if arrBin[0] == 1 and arrBin[6] == 1: n[5] += 1
    
    return n

def arrayToBin(arr):
    newArr = [0, 0, 0, 0, 0, 0]
    iNewArr = 0
    for x in arr:
        if x != 0:
            newArr[iNewArr] = 1
        iNewArr+=1
    return newArr

iguais = {}

for combination in itertools.product(range(2), repeat=6):
    iguais[''.join(map(str, combination))] = {"users":[]}

for key, value in list(pacientes.items()):
    visitas = [[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],
               [0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' '],[0, ' ', ' ', ' ', ' ']]
    #print(visitas)
    for exame in value["exames"]:
        visita = {"id":1,"idade":exame["age"],"mmse":exame["mmse"], "cdr":exame["cdr"], 
                  "imagem-id":exame["image-id"]}
        
        if(exame["dmes"] == "6"): 
            visitas[1] = list(visita.values())
        elif(exame["dmes"] == "12"): 
            visitas[2] = list(visita.values())     
        elif(exame["dmes"] == "18"): 
            visitas[3] = list(visita.values())
        elif(exame["dmes"] == "24"): 
            visitas[4] = list(visita.values())
        elif(exame["dmes"] == "36"): 
            visitas[5] = list(visita.values())
        elif(exame["dmes"] == "48"): 
            visitas[6] = list(visita.values())
        else:
            visitas[0] = list(visita.values())
    
    transposta = list(map(lambda *i: [j for j in i], *visitas))
    transpostaEmLinha = []
    for x in transposta:
        transpostaEmLinha+=x
    
    arrToPrint = [visitas[0][0], visitas[1][0], visitas[2][0], visitas[3][0], visitas[4][0], visitas[5][0],
                 visitas[6][0]]
    
    combinacoes = contaDelta(arrToPrint)
    combinacoesBin = arrayToBin(combinacoes)
    combinacoesBinStr = ''.join(map(str, combinacoesBin))
    iguais[combinacoesBinStr]["users"].append(key)
    
#print(pacientes.keys())
#print(combinacoesBinStr)
  
print(iguais)
#for ig in iguais:
#    print(ig + ': ' + str(iguais[ig]))

In [ ]:
#PROCESS DATA FROM 10 CSV's AFTER FREESURFER SEGMENTATION

colunas = ["Paciente"]
pacientes = {}

soma = 0

listaDeFicheiros = ["aseg_volume_stats.csv", "aseg_mean_stats.csv", "aparc_volume_rh.csv", "aparc_volume_lh.csv",
                    "aparc_thickness_rh.csv", "aparc_thickness_lh.csv", "aparc_meancurv_rh.csv", "aparc_meancurv_lh.csv",
                    "aparc_area_rh.csv", "aparc_area_lh.csv", "wmparc_stats.csv"]

id_Paciente = 0
for ficheiro in listaDeFicheiros:
    with open(ficheiro, 'r') as read_obj:
        csv_reader = reader(read_obj)
        for collumn in csv_reader:
            i = 0
            for coluna in collumn:
                if ficheiro == "aseg_volume_stats.csv":
                    coluna = coluna + "_volume"
                if ficheiro == "aseg_mean_stats.csv":
                    coluna = coluna + "_mean"
                if i != 0:
                    colunas.append(coluna)
                i+=1
                soma +=1
            
            break
        for row in csv_reader:
            nomePaciente = row[0][5:15]
            if nomePaciente in pacientes.keys():
                pacientes[nomePaciente].extend(row[1:])
            else:
                pacientes[nomePaciente] = list(row[1:])
            id_Paciente+=1



df = pd.DataFrame([],columns=colunas)
for paciente in pacientes:
    lista = [paciente]
    lista.extend(pacientes[paciente])
    #print(lista)
    df.loc[df.shape[0]] = lista 

df.to_csv('Final.csv', index=False)